In [1]:
import os
os.chdir('..')
os.getcwd()

'/Users/signapoop/Desktop/fyp-graph-clustering'

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt

import pickle 
import numpy as np
import pandas as pd
import torch
import os
from timeit import default_timer as timer
import networkx as nx
import scipy.sparse as sp

from sklearn.manifold import TSNE
from sklearn.metrics.pairwise import pairwise_distances

In [3]:
from core.SimpleNet import SimpleNet
from core.OldGraphConvNet import OldGraphConvNet
from core.GraphConvNet import GraphConvNet
from core.EmbeddingDataSet import EmbeddingDataSet
from util.plot_graph_embedding import plot_graph_embedding
from util.evaluation_metrics import get_net_projection, neighborhood_preservation, trustworthiness, evaluate_net_metrics
from util.graph_utils import get_shortest_path_matrix

In [4]:
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [5]:
if torch.cuda.is_available():
    print('cuda available')
    device = 'cuda'
else:
    print('cuda not available')
    device = 'cpu'

cuda not available


In [6]:
dataset_name = 'cora'
parent_dir = os.path.abspath('..')
input_dir = os.path.join(parent_dir, 'data')
dataset = EmbeddingDataSet(dataset_name, input_dir, train=True)

In [7]:
net_parameters = {}
net_parameters['n_components'] = 2
net_parameters['D'] = dataset.input_dim # input dimension
net_parameters['H'] = 50 # number of hidden units
net_parameters['L'] = 2 # number of hidden layers

In [8]:
# Graph net
net_1 = GraphConvNet(net_parameters)
if torch.cuda.is_available(): net_1.cuda()
root = 'results/cora_27/'
filename = root + 'graph_net_600.pkl'
checkpoint = torch.load(filename, map_location=device)
net_1.load_state_dict(checkpoint['state_dict'])

In [9]:
dataset = EmbeddingDataSet(dataset_name, input_dir, train=True)
dataset.create_all_data(n_batches=1, shuffle=False)
y_pred_1 = get_net_projection(dataset.all_data, net_1)
y_pred_1 = y_pred_1[np.argsort(dataset.all_indices)]
plot_graph_embedding(y_pred_1, dataset.labels, dataset.adj_matrix)

In [10]:
dataset = EmbeddingDataSet(dataset_name, input_dir, train=False)
dataset.create_all_data(n_batches=1, shuffle=False)
y_pred_1 = get_net_projection(dataset.all_data, net_1)
y_pred_1 = y_pred_1[np.argsort(dataset.all_indices)]
plot_graph_embedding(y_pred_1, dataset.labels, dataset.adj_matrix)

In [12]:
dataset = EmbeddingDataSet('cora_full', input_dir, train=True)
dataset.create_all_data(n_batches=1, shuffle=False)
y_pred_1 = get_net_projection(dataset.all_data, net_1)
y_pred_1 = y_pred_1[np.argsort(dataset.all_indices)]
plot_graph_embedding(y_pred_1, dataset.labels, dataset.adj_matrix)